# Exercise 5: Lasso Regression on the Franke function  with resampling (Score 10 points))


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import os
from common import *
import pandas as pd
#from mpl_toolkits.mplot3d import Axes3D
import sklearn.linear_model as lm
from sklearn.model_selection import train_test_split
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

print(f"Root directory: {os.getcwd()}")

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "font.size": 10,
})

%matplotlib inline 

# 1.5
Start off by defining the data

In [ ]:
SEED = np.random.seed(4155)
maxdegree = 12
n = 20
x = np.sort(np.random.uniform(0, 1, n))
y = np.sort(np.random.uniform(0, 1, n))
x,y = np.meshgrid(x,y)
t = FrankeFunction(x, y) + noise_factor(n, factor=0.2)
min_lambda = -9
max_lambda = 4
nlambdas = 100
lambdas = np.logspace(min_lambda, max_lambda, nlambdas)

We want to essentially repeat the previous two exercises, as such, this exercise will follow in the footsteps of exercise 4. The difference is that the sci-kit learn implementation of Lasso regression is used. 

# Create a searchlandscape

In [ ]:
test_mse = np.zeros((maxdegree, nlambdas))
train_mse = np.zeros_like(test_mse)
model_list = np.empty_like(test_mse, dtype=object)
optimal_deg = 0
optimal_lmb = 0
lambda_degree = 0
best_mse = np.inf

for deg in range(1, maxdegree+1):
    for lmb in range(len(lambdas)):

        X = create_X(x,y,n=deg)

        X_train, X_test, z_train, z_test = prepare_data(X, t.ravel(), 4155, scale_X=True, skip_intercept=True)

        model = lm.Lasso(lambdas[lmb], fit_intercept=False)
        model.fit(X_train, z_train)
        z_hat_train = model.predict(X_train)
        z_hat_test = model.predict(X_test)

        test_mse[deg-1,lmb] = MSE(z_test, z_hat_test)
        train_mse[deg-1, lmb] = MSE(z_train, z_hat_train)
        model_list[deg-1,lmb] = model
        
        if test_mse[deg-1,lmb] < best_mse:
            best_mse = test_mse[deg-1, lmb]
            optimal_deg = deg
            optimal_lmb = lambdas[lmb]
            lambda_degree = lmb

print(best_mse)
print(optimal_lmb)
print(optimal_deg)
print(train_mse[deg-1, lmb])
optimal_model = model_list[deg-1, lambda_degree]

In [ ]:
import matplotlib.ticker as mticker
%matplotlib
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.title.set_text("Plot of the Search Landscape")
ax.set_xlabel("Polynomial degree"); ax.set_ylabel("Lambda index"); ax.set_zlabel("MSE")
#ax.set_yticklabels(f"{np.log10(lambdas[i])}" for i in np.linspace(0,499,6,dtype=int))
ax.set_xticklabels(f"{deg-2}" for deg in range(1, maxdegree+2,2))

degs, lambs = np.meshgrid(range(maxdegree), range(nlambdas))
print(degs.shape)
print(lambs.shape)
print(test_mse.shape)
surf = ax.plot_surface(degs, lambs, test_mse.swapaxes(0,1), cmap=cm.coolwarm)
ax.view_init(elev=14., azim=-58.)
ax.scatter(optimal_deg - 1, lambda_degree, best_mse, c='r', marker='o', s=100)
plt.savefig(f"{REPORT_FIGURES}{EX5}search_landscape_lasso.pdf")

# Betaplot showing effect of L1 regularizer

In [ ]:
%matplotlib inline
for deg in range(1, maxdegree-3):     # Only want the first 8
    betas = []
    summaries_df = pd.DataFrame()
    for lmb in lambdas:
        X = create_X(x,y,n=deg)
        X_train, X_test, z_train, z_test = prepare_data(X, t.ravel(), 4155, scale_X=True, skip_intercept=True)
        model = lm.Lasso(lmb, fit_intercept=False)
        model.fit(X_train, z_train)
        betas.append(model.get_params())
        
    #fig = plot_beta_errors_for_lambdas(summaries_df, deg)
    #plt.savefig(f"{REPORT_FIGURES}{EX4}beta_plot_ridge_{deg}.pdf")

# Cross Validation